In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import sys, os

sys.path.append(os.path.abspath(os.path.join("../..")))
sys.path.append(os.path.abspath(os.path.join("../scripts")))

In [3]:
import eda_functions as eda

In [4]:
import dvc.api

In [5]:
from urllib.parse import urlparse

In [6]:
import warnings 

# Suppressing warning messages
warnings.filterwarnings('ignore')

In [7]:
# set font size 

plt.rcParams.update({'font.size': 12})

# Read The Data

In [8]:
#path = 'data/store.csv'
#repo = 'https://github.com/emtinanseo/Sales-Future-Forecast.git'
#version = 'v0'

#data_url = dvc.api.get_url(path=path, repo=repo, rev=version)

In [9]:
# import the store data: store.csv - supplemental information about the stores

data_store = pd.read_csv('../data/store.csv')

In [10]:
#path = 'data/train.csv'
#repo = 'https://github.com/emtinanseo/Sales-Future-Forecast.git'
#version = 'v0'

#data_url = dvc.api.get_url(path=path, repo=repo, rev=version)

In [11]:
# import the train data: train.csv - historical data including Sales

data_train = pd.read_csv('../data/train.csv')

In [12]:
#path = 'data/test.csv'
#repo = 'https://github.com/emtinanseo/Sales-Future-Forecast.git'
#version = 'v0'

#data_url = dvc.api.get_url(path=path, repo=repo, rev=version)

In [13]:
# import the test data: test.csv - historical data excluding Sales

data_test = pd.read_csv('../data/test.csv')

# Exploring Train and Test Data

## Size

In [14]:
print("train data has {} rows and {} columns".format(data_train.shape[0],data_train.shape[1]))

train data has 1017209 rows and 9 columns


In [15]:
print("test data has {} rows and {} columns".format(data_test.shape[0],data_test.shape[1]))

test data has 41088 rows and 8 columns


In [16]:
total = data_train.shape[0] + data_test.shape[0]

print("of the total data, {:.2f}% is train and {:.2f}% is test".format(data_train.shape[0]*100/total,
                                                                      data_test.shape[0]*100/total))

of the total data, 96.12% is train and 3.88% is test


## Summary

In [17]:
eda.missing_values_table(data_train)

AttributeError: module 'eda_functions' has no attribute 'missing_values_table'

In [ ]:
missing_values_table